<a href="https://colab.research.google.com/github/krispy11tw/250416/blob/main/250430.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import urllib.request as req
import bs4 as bs

# 先定義好函式:
# save_post_meta
def save_post_meta(url):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1745582404.A.011.html"
    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
    resp = req.urlopen(r)
    html = bs.BeautifulSoup(resp.read())
    # 作者/看版/標題/日期
    metas = html.find_all("span", {"class":"article-meta-value"})
    uid_text = metas[0].text
    board_text = metas[1].text
    title_text = metas[2].text
    time_text = metas[3].text
    # print(uid_text, board_text, title_text, time_text, sep="\n")
    # 每個推虛文 [{"type":"", "id":"", "text":"", "date_ip":""}]
    # JSON格式: {"左者": "看板": "表提": "推虛文":[]}
    push_list = []
    pushes = html.find_all("div", {"class":"push"})
    for push in pushes:
        push_meta = push.find_all("span")
        # 推噓處理
        push_type_text = push_meta[0].text
        if "推" in push_type_text:
            push_type_text = 1
        elif "噓" in push_type_text:
            push_type_text = -1
        else:
            push_type_text = 0
        # 推噓id
        push_id_text = push_meta[1].text
        # 推噓內容
        push_content_text = push_meta[2].text.replace(": ", "")
        # 推噓ipdate
        push_ipdate_text = push_meta[3].text.strip()
        # print(push_type_text, push_id_text, push_content_text, push_ipdate_text, sep="\t")
        push_data = {
            "type":push_type_text,
            "id":push_id_text,
            "text":push_content_text,
            "date_ip":push_ipdate_text
        }
        push_list.append(push_data)
    # print(push_list)
    post = {
        "作者":uid_text,
        "看板":board_text,
        "標題":title_text,
        "日期":time_text,
        "推噓文":push_list,
    }
    # print(post)
    fn = url.split("/")[-1] + ".json"
    with open(fn, "w", encoding="utf-8") as f:
        json.dump(post, f, ensure_ascii=False, indent=4)

`````
當你發現你沒有class/id做篩選的時候
1. 找他老爸
2. 自己想辦法篩選
`````

In [4]:
import json
import urllib.request as req
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/M.1745582404.A.011.html"
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
resp = req.urlopen(r)
html = bs.BeautifulSoup(resp.read())

allow_sub = ["jpg", "jpeg", "png", "gif"]
link_list = html.find_all("a")
for link in link_list:
    link_href = link["href"]
    sub = link_href.split(".")[-1]
    # lower是字串的專屬功能
    if sub.lower() in allow_sub:
        print(link_href)

https://i.imgur.com/S20cVi5.jpeg
https://i.imgur.com/drwno1X.jpeg
https://i.imgur.com/BcT3DCj.jpeg
https://i.imgur.com/lSsIOH6.jpeg
https://i.imgur.com/ZlZXDz3.jpeg
https://i.imgur.com/A84pbtB.jpeg
https://i.imgur.com/jxBcoCS.jpeg
https://i.imgur.com/aqvICAJ.jpeg
https://i.imgur.com/4E9ZS1L.jpeg
https://i.imgur.com/QKtIEwK.jpeg
https://i.imgur.com/Dfdvviy.jpeg
https://i.meee.com.tw/hNil3HB.gif
https://i.meee.com.tw/ADxUv1E.gif


In [7]:

# os.path這件事我們後面會解釋
import os
# 如果資料夾不存在, 就創起來
if not os.path.exists("a/b"):
    os.makedirs("a/b")

img_url = "https://i.imgur.com/S20cVi5.jpeg"
# 如果對面什麼都沒檢查, 最簡單的一訂是urlretrieve
r = req.Request(img_url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
img_resp = req.urlopen(r)
img_content = img_resp.read()
# 存檔的時候 你要開啟檔案在本地端, 把剛才的內容寫進去
# 之前教你open的時候, 第二個參數 w r  + encoding="utf-8" -> for 純文字
#                              wb rb -> for 非純文字
with open("a/b/test.png", "wb") as f:
    f.write(img_content)

In [8]:


import json
import urllib.request as req
import bs4 as bs
import os

url = "https://www.ptt.cc/bbs/Beauty/M.1745582404.A.011.html"
# prepare directory
dn = url.split("/")[-1]
if not os.path.exists(dn):
    os.makedirs(dn)

r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
resp = req.urlopen(r)
html = bs.BeautifulSoup(resp.read())

allow_sub = ["jpg", "jpeg", "png", "gif"]
link_list = html.find_all("a")
for link in link_list:
    link_href = link["href"]
    sub = link_href.split(".")[-1]
    # lower是字串的專屬功能
    if sub.lower() in allow_sub:
        img_url = link_href
        # 如果對面什麼都沒檢查, 最簡單的一訂是urlretrieve
        r = req.Request(img_url)
        r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
        img_resp = req.urlopen(r)
        img_content = img_resp.read()
        # 存檔的時候 你要開啟檔案在本地端, 把剛才的內容寫進去
        # 之前教你open的時候, 第二個參數 w r  + encoding="utf-8" -> for 純文字
        #                              wb rb -> for 非純文字
        fn = dn + "/" + img_url.split("/")[-1]
        with open(fn, "wb") as f:
            f.write(img_content)

In [18]:
import json
import urllib.request as req
import bs4 as bs
import os

# 先定義好函式:
# save_post_img

def save_post_img(url):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1745582404.A.011.html"
    # prepare directory
    dn = url.split("/")[-1]
    if not os.path.exists(dn):
        os.makedirs(dn)

    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
    resp = req.urlopen(r)
    html = bs.BeautifulSoup(resp.read())

    allow_sub = ["jpg", "jpeg", "png", "gif"]
    link_list = html.find_all("a")
    for link in link_list:
        link_href = link["href"]
        sub = link_href.split(".")[-1]
        # lower是字串的專屬功能
        if sub.lower() in allow_sub:
            img_url = link_href
            # 如果對面什麼都沒檢查, 最簡單的一訂是urlretrieve
            r = req.Request(img_url)
            r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
            img_resp = req.urlopen(r)
            img_content = img_resp.read()
            # 存檔的時候 你要開啟檔案在本地端, 把剛才的內容寫進去
            # 之前教你open的時候, 第二個參數 w r  + encoding="utf-8" -> for 純文字
            #                              wb rb -> for 非純文字
            fn = dn + "/" + img_url.split("/")[-1]
            with open(fn, "wb") as f:
                f.write(img_content)

In [19]:

#練習:獲取ptt表特版的一整頁網址

import json
import urllib.request as req
import bs4 as bs
import os

url = "https://www.ptt.cc/bbs/Beauty/index4000.html"
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
resp = req.urlopen(r)
html = bs.BeautifulSoup(resp.read())

div_list = html.find_all("div", {"class":"title"})
for div in div_list:
    post_link = div.find("a")
    if not post_link == None:
        post_link_url = "https://www.ptt.cc" + post_link["href"]
        print(post_link_url)
        save_post_meta(post_link_url)
        save_post_img(post_link_url)

https://www.ptt.cc/bbs/Beauty/M.1745759627.A.BA1.html
https://www.ptt.cc/bbs/Beauty/M.1745763958.A.745.html
https://www.ptt.cc/bbs/Beauty/M.1745785036.A.560.html
https://www.ptt.cc/bbs/Beauty/M.1745785622.A.395.html
https://www.ptt.cc/bbs/Beauty/M.1745785821.A.BAC.html
https://www.ptt.cc/bbs/Beauty/M.1745807999.A.1D6.html
https://www.ptt.cc/bbs/Beauty/M.1745808444.A.680.html
https://www.ptt.cc/bbs/Beauty/M.1745816387.A.7E2.html
https://www.ptt.cc/bbs/Beauty/M.1745817760.A.93F.html
https://www.ptt.cc/bbs/Beauty/M.1745822327.A.B6D.html
https://www.ptt.cc/bbs/Beauty/M.1745824168.A.272.html
https://www.ptt.cc/bbs/Beauty/M.1745829911.A.9DC.html
https://www.ptt.cc/bbs/Beauty/M.1745835734.A.581.html
https://www.ptt.cc/bbs/Beauty/M.1745839803.A.B32.html
https://www.ptt.cc/bbs/Beauty/M.1745842649.A.10F.html
https://www.ptt.cc/bbs/Beauty/M.1745843241.A.FF8.html
https://www.ptt.cc/bbs/Beauty/M.1745843920.A.C44.html
https://www.ptt.cc/bbs/Beauty/M.1745846671.A.EB9.html


In [ ]:
# 如果你要把雲端硬碟當成隨身碟


# dn = "drive/MyDrive/tiba0430"
# if not os.path.exists(dn):
#    os.makedirs(dn)
# fn = dn + "/" + "a.txt"
# with open(fn, "w", encoding="utf-8") as f:
#    f.write("abcde")

In [ ]:
# 如果要將資料夾以壓縮檔方式下載
# 使用命令列
# !zip -r /content/file.zip + 檔案路徑